In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [5]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [6]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [7]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [8]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_time_loss(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_time_loss(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(270)
train_Y = user_function_time_loss(train_X)

[[18.         22.78911208  6.63935858 54.19967041  2.          2.55857348]
 [ 7.         15.27093135 12.32432809 39.05614891  1.          3.31556836]
 [12.          9.06669986 25.68186945 52.4123406   2.          4.56258435]
 [17.         17.9305433  24.89615228 61.57427586  2.          3.80957295]
 [14.         19.0021036  22.44166005 38.76229273  1.          3.13751616]
 [13.         18.69157052 25.76700268 68.51499285  3.          2.93306007]
 [ 9.         21.06271216  5.87407525 59.44067738  2.          4.98559951]
 [10.         23.78793233 17.59953708 42.91597458  3.          2.05725981]
 [19.         12.41298343 22.22918169 32.10938704  3.          1.76951929]
 [ 3.          9.50892476  5.66519584 49.80634504  2.          4.27472206]
 [15.         12.75362243 25.67356262 53.35041656  2.          3.30788967]
 [13.         13.02747163 22.24663498 53.77147656  2.          3.10806111]
 [12.         14.99884902 16.73085885 55.96170769  3.          2.165224  ]
 [10.         17.36780815


Evaluating input: 36 of 50

 Retrying in 1 seconds

Evaluating input: 37 of 50

 Retrying in 1 seconds

Evaluating input: 38 of 50

 Retrying in 1 seconds

Evaluating input: 39 of 50

 Retrying in 1 seconds

Evaluating input: 40 of 50

 Retrying in 1 seconds

Evaluating input: 41 of 50

 Retrying in 1 seconds

Evaluating input: 42 of 50

 Retrying in 1 seconds

Evaluating input: 43 of 50

 Retrying in 1 seconds

Evaluating input: 44 of 50

 Retrying in 1 seconds

Evaluating input: 45 of 50

 Retrying in 1 seconds

Evaluating input: 46 of 50

 Retrying in 1 seconds

Evaluating input: 47 of 50

 Retrying in 1 seconds

Evaluating input: 48 of 50

 Retrying in 1 seconds

Evaluating input: 49 of 50

 Retrying in 1 seconds

Evaluating input: 50 of 50

 Retrying in 1 seconds
[[15.          9.58883366 22.81123894 37.97311593  3.          4.47357999]
 [16.         14.60743304 10.06974995 32.07998251  3.          1.74326609]
 [11.         20.67346781 11.54124217 45.47143814  2.          2.43013


Evaluating input: 131 of 270

 Retrying in 1 seconds

Evaluating input: 132 of 270

 Retrying in 1 seconds

Evaluating input: 133 of 270

 Retrying in 1 seconds

Evaluating input: 134 of 270

 Retrying in 1 seconds

Evaluating input: 135 of 270

 Retrying in 1 seconds

Evaluating input: 136 of 270

 Retrying in 1 seconds

Evaluating input: 137 of 270

 Retrying in 1 seconds

Evaluating input: 138 of 270

 Retrying in 1 seconds

Evaluating input: 139 of 270

 Retrying in 1 seconds

Evaluating input: 140 of 270

 Retrying in 1 seconds

Evaluating input: 141 of 270

 Retrying in 1 seconds

Evaluating input: 142 of 270

 Retrying in 1 seconds

Evaluating input: 143 of 270

 Retrying in 1 seconds

Evaluating input: 144 of 270

 Retrying in 1 seconds

Evaluating input: 145 of 270

 Retrying in 1 seconds

Evaluating input: 146 of 270

 Retrying in 1 seconds

Evaluating input: 147 of 270

 Retrying in 1 seconds

Evaluating input: 148 of 270

 Retrying in 1 seconds

Evaluating input: 149 of 27

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [9]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = -22.106309144390245
Optimization restart 2/5, f = 3.7397078516329305
Optimization restart 3/5, f = 3.7397078516357496
Optimization restart 4/5, f = 3.739707851579622
Optimization restart 5/5, f = -22.10630914438997


GP_regression.,value,constraints,priors
rbf.variance,0.09324678711392748,+ve,
rbf.lengthscale,26.569969944880206,+ve,
Gaussian_noise.variance,0.0007940475117961487,+ve,


#### Acquisition Function

In [10]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [11]:
optimizer = GradientAcquisitionOptimizer(parameter_space)


#### Experimental Design Loop

In [12]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [13]:
ed_loop.run_loop(user_function_time_loss, 250)

Optimization restart 1/5, f = -22.106309144390245
Optimization restart 2/5, f = 3.7397078516329305
Optimization restart 3/5, f = -22.106309144353034
Optimization restart 4/5, f = 3.7397078516440487
Optimization restart 5/5, f = -22.106309138518387
[[ 3.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -22.302192391265784
Optimization restart 2/5, f = -22.302192390417634
Optimization restart 3/5, f = 3.4659891994882948
Optimization restart 4/5, f = -22.302192379415366
Optimization restart 5/5, f = 3.4659891995137944
[[ 3.   8.  27.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -22.947883366553388
Optimization restart 2/5, f = -22.947883366541895
Optimization restart 3/5, f = 3.4235159386747007
Optimization restart 4/5, f = 3.42351593867917
Optimization restart 5/5

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


 Retrying in 1 seconds
Optimization restart 1/5, f = -27.11035471014295
Optimization restart 2/5, f = 3.860171986434299
Optimization restart 3/5, f = 3.8601767951413457
Optimization restart 4/5, f = 3.8601651204937415
Optimization restart 5/5, f = 3.860173280894811
[[20.  25.  27.  30.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -27.245470094283263
Optimization restart 2/5, f = -27.245470094655886
Optimization restart 3/5, f = -27.24547009465366
Optimization restart 4/5, f = 5.8739720481757605
Optimization restart 5/5, f = 5.873972046029381
[[ 3.   8.   5.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -28.34033012416145
Optimization restart 2/5, f = 5.642860602464001
Optimization restart 3/5, f = -28.340330124284947
Optimization restart 4/5, f = -28.340330124285018
Optimiz

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


 Retrying in 1 seconds
Optimization restart 1/5, f = -31.118134137740267
Optimization restart 2/5, f = -31.118134137756947
Optimization restart 3/5, f = -31.11813413775006
Optimization restart 4/5, f = 7.283810572621132
Optimization restart 5/5, f = -31.11813413775873
[[20.          8.          5.         47.89180892  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -32.075372055826605
Optimization restart 2/5, f = 7.344936893735273
Optimization restart 3/5, f = -32.07537205430241
Optimization restart 4/5, f = 7.3449367198931625
Optimization restart 5/5, f = 7.344936893731418
[[ 3.         25.         17.92841205 70.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -33.88697055119516
Optimization restart 2/5, f = -33.88697055121459
Optimization restart 

...

Optimization restart 3/5, f = -264.58471802984286
Optimization restart 4/5, f = -238.4587110295406
Optimization restart 5/5, f = -238.45871059232056
[[20.         25.         17.77901904 30.          3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -265.6319289644282
Optimization restart 2/5, f = -239.49928718732787
Optimization restart 3/5, f = -239.49928547626956
Optimization restart 4/5, f = -239.49919466955453
Optimization restart 5/5, f = -239.49928742972665
[[ 3.         17.36803197 27.         30.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -267.2381190463238
Optimization restart 2/5, f = -241.15968529089747
Optimization restart 3/5, f = -241.15969120939377
Optimization restart 4/5, f = -267.23811904825664
Optimization restart 5/5, f = -241.

Optimization restart 5/5, f = -275.46017536204744
[[ 3.  8.  5. 30.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -298.9512893600262
Optimization restart 2/5, f = -274.41952092657795
Optimization restart 3/5, f = -274.4195203956084
Optimization restart 4/5, f = -274.419519987752
Optimization restart 5/5, f = -274.41951899696625
[[14.         13.44350625  5.         70.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -301.2525960340348
Optimization restart 2/5, f = -277.0629869386065
Optimization restart 3/5, f = -277.06311365455355
Optimization restart 4/5, f = -277.06311367771843
Optimization restart 5/5, f = -277.0631136778795
[[12.        25.        27.        40.4609318  1.         1.5      ]]

User function time loss called with 1 inputs to simulate

Evaluati

 Retrying in 1 seconds
Optimization restart 1/5, f = -326.5466402944253
Optimization restart 2/5, f = -271.7528360577147
Optimization restart 3/5, f = -271.75306988641114
Optimization restart 4/5, f = -326.5467326782584
Optimization restart 5/5, f = -271.7530751337745
[[ 3.         25.          5.         40.25914954  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -328.8353596145979
Optimization restart 2/5, f = -273.7009749603001
Optimization restart 3/5, f = -273.7009776192416
Optimization restart 4/5, f = -273.7009781828091
Optimization restart 5/5, f = -273.70097818298336
[[20.         18.55319532 15.06375037 70.          1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -330.5848256164804
Optimization restart 2/5, f = -275.97381819265337
Optimization resta

Optimization restart 2/5, f = -314.84389232976207
Optimization restart 3/5, f = -361.9143846342491
Optimization restart 4/5, f = -361.914384721991
Optimization restart 5/5, f = -314.84309883162837
[[20.         25.         27.         54.18098789  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -364.3006061887793
Optimization restart 2/5, f = -364.3006061944408
Optimization restart 3/5, f = -364.300606195183
Optimization restart 4/5, f = -317.47944388593595
Optimization restart 5/5, f = -317.47944388786095
[[ 3.         25.         19.64475009 37.0200167   3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -366.690522003715
Optimization restart 2/5, f = -319.2729798757874
Optimization restart 3/5, f = -319.27297808221573
Optimization restart 4/5, f = -319.2729798

Optimization restart 3/5, f = -401.3487008777911
Optimization restart 4/5, f = -401.3487008707281
Optimization restart 5/5, f = -340.94613576434966
[[12.         25.         12.20508415 58.04035445  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -403.05140202089547
Optimization restart 2/5, f = -343.1173322439935
Optimization restart 3/5, f = -343.11733271139076
Optimization restart 4/5, f = -343.1173297060557
Optimization restart 5/5, f = -343.11733271131834
[[11.         18.48822497 27.         48.64616672  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -405.50704865479173
Optimization restart 2/5, f = -405.5071357162651
Optimization restart 3/5, f = -345.0637110657958
Optimization restart 4/5, f = -405.5071357092812
Optimization restart 5/5, f = -345.0637

### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [14]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = -22.106309144390245
Optimization restart 2/5, f = -22.106309144380933
Optimization restart 3/5, f = -22.10630914436508
Optimization restart 4/5, f = 3.7397078512365187
Optimization restart 5/5, f = -22.106309144381058


#### Acquisition Function

In [15]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ird, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [16]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [17]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [18]:
ed_loop.run_loop(user_function_time_loss, 250)

Optimization restart 1/5, f = -22.106309144390245
Optimization restart 2/5, f = 3.739707851632925
Optimization restart 3/5, f = 3.7397078516436757
Optimization restart 4/5, f = -19.395770869430272
Optimization restart 5/5, f = -22.106309144392547


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[[ 4.         21.07399884 20.897432   57.87949848  1.          1.63553464]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -22.01546717120799
Optimization restart 2/5, f = 5.236329265753941
Optimization restart 3/5, f = 5.236329399312638
Optimization restart 4/5, f = 5.236329399313188
Optimization restart 5/5, f = -22.015467171232217
[[ 3.         13.69671901 11.01143721 68.41074992  3.          2.25529686]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -21.641311190043083
Optimization restart 2/5, f = -21.641311191301323
Optimization restart 3/5, f = -21.641311191376886
Optimization restart 4/5, f = 5.153813803075472
Optimization restart 5/5, f = 5.153813652915208
[[16.         23.65507373 10.99173907 31.34139184  2.          1.60375812]]

User function time loss called with 1 inputs to simulate

Evaluati

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[[ 3.          8.83551429 20.09681223 37.2946094   2.          2.48730874]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -27.258413440841327
Optimization restart 2/5, f = 6.357247938092025
Optimization restart 3/5, f = -27.25841344079972
Optimization restart 4/5, f = 6.357240865755422
Optimization restart 5/5, f = 6.357246969411651
[[ 3.         21.10354188  5.03097812 37.55040538  2.          1.5580608 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -28.627695961480043
Optimization restart 2/5, f = -28.627695961452055
Optimization restart 3/5, f = -28.627695961504884
Optimization restart 4/5, f = -28.627695961506962
Optimization restart 5/5, f = 6.1819484036492796
[[18.         23.02431549 24.97317915 56.04432437  2.          3.88107179]]

User function time loss called with 1 inputs to simulate

Evalu

...

Optimization restart 4/5, f = -420.6451097966411
Optimization restart 5/5, f = -431.1956052761299
[[12.         24.66083186  5.40809778 30.30062632  1.          1.89451762]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -433.5310517340698
Optimization restart 2/5, f = -433.53108089256
Optimization restart 3/5, f = -433.5310809029677
Optimization restart 4/5, f = -433.5310809027312
Optimization restart 5/5, f = -433.5310808917611
[[20.          9.59938204 26.59945212 54.19179287  3.          2.42472608]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -435.93853295018437
Optimization restart 2/5, f = 64.28304558608517
Optimization restart 3/5, f = -435.93859278858247
Optimization restart 4/5, f = -435.9385927913573


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = -435.938592791812
[[ 4.          8.16872311  9.41042547 58.26242897  2.          1.60416647]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -437.32353333950147
Optimization restart 2/5, f = -437.32353333947106
Optimization restart 3/5, f = -437.3235333380818
Optimization restart 4/5, f = -437.3235333395365
Optimization restart 5/5, f = -437.32353331168525
[[20.         12.36900833 26.72808957 42.06150878  1.          1.91211845]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -439.7284954543816
Optimization restart 2/5, f = -439.7285056673113
Optimization restart 3/5, f = -439.72850569786766
Optimization restart 4/5, f = -439.7285056980203
Optimization restart 5/5, f = -439.7285056954928
[[14.         11.03531306 18.73988747 37.83424447  1.          4.79685058]]

User function

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -448.1462566447401
Optimization restart 3/5, f = -450.72132389526155
Optimization restart 4/5, f = 65.36829775059178
Optimization restart 5/5, f = -450.7213238934057
[[ 3.         18.37751285 22.31087442 46.83623661  3.          1.61033774]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -452.57416905892273
Optimization restart 2/5, f = -452.5741690598395
Optimization restart 3/5, f = -452.57416905985167
Optimization restart 4/5, f = -452.5741690629795
Optimization restart 5/5, f = -452.57416906316695


### Random GP

Supervised learning

In [19]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=50)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/50, f = -512.2030638998212
Optimization restart 2/50, f = -512.2030640286398
Optimization restart 3/50, f = -512.2030640290843
Optimization restart 4/50, f = -512.2030640201525
Optimization restart 5/50, f = -512.2030640289969
Optimization restart 6/50, f = -512.203063792999
Optimization restart 7/50, f = 55.18206825175798
Optimization restart 8/50, f = -512.203064028459
Optimization restart 9/50, f = -512.203063651166
Optimization restart 10/50, f = 55.18206854534654
Optimization restart 11/50, f = -512.2030623774881
Optimization restart 12/50, f = 55.182058291435965
Optimization restart 13/50, f = -512.2030640290802
Optimization restart 14/50, f = 55.182068514598114
Optimization restart 15/50, f = -512.2030640288672
Optimization restart 16/50, f = -512.203064029027
Optimization restart 17/50, f = -512.2030639699112
Optimization restart 18/50, f = -512.2030640268636
Optimization restart 19/50, f = 55.182068545346624


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 20/50, f = -512.2030640290823
Optimization restart 21/50, f = -512.203064007515
Optimization restart 22/50, f = -512.2030640290832
Optimization restart 23/50, f = -512.2030640263912
Optimization restart 24/50, f = -512.2030640289362
Optimization restart 25/50, f = -512.2030640279381
Optimization restart 26/50, f = -512.2030640220779
Optimization restart 27/50, f = -512.203064028738
Optimization restart 28/50, f = 55.18206854534375
Optimization restart 29/50, f = -512.2030640290834
Optimization restart 30/50, f = -512.2030640255547
Optimization restart 31/50, f = -512.2030640290831
Optimization restart 32/50, f = -512.2030640288842
Optimization restart 33/50, f = 55.18205657010873
Optimization restart 34/50, f = 55.182068545346596
Optimization restart 35/50, f = -512.203063925633
Optimization restart 36/50, f = -512.2030640202236
Optimization restart 37/50, f = -512.2030640290328
Optimization restart 38/50, f = -512.2030640290849
Optimization restart 39/50, f = -512

GP_regression.,value,constraints,priors
rbf.variance,0.053360580905509325,+ve,
rbf.lengthscale,16.70109427072523,+ve,
Gaussian_noise.variance,0.0006412101135249977,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [20]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  0.031421356052996174


In [21]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  0.03031180445853298


In [22]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  0.03308872848686157


In [23]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is 5.039% better than training on random points.'

In [24]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is 8.392% better than training on random points.'

### Save Models

In [25]:
with open(f'experimental_design/models/model_variance_20_init_points_250_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design/models/integrated_variance_reduction_20_init_points_250_loops_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design/models/270_random_points_supervised_GP_timeloss.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)